<a href="https://colab.research.google.com/github/notyourfriends/Playground/blob/main/Tech_Layoffs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load the data

In [79]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [80]:
import pandas as pd

In [81]:
data = pd.read_csv('/content/drive/MyDrive/CSVs/tech_layoffs.csv')

In [82]:
data.columns

Index(['company', 'total_layoffs', 'impacted_workforce_percentage',
       'reported_date', 'industry', 'headquarter_location', 'sources',
       'status', 'additional_notes'],
      dtype='object')

In [83]:
data.head(5)

,company,total_layoffs,impacted_workforce_percentage,reported_date,industry,headquarter_location,sources,status,additional_notes
0,C2FO,20,2,12/9/2022,"Fintech, payments","Leawood, KS",Kansas City Business Journal,Private,NaN
1,VideoAmp,Unclear,2,9/19/2022,Advertising platforms,Los Angeles,Company memo,Private,NaN
2,Amperity,13,3,8/20/2022,"Saas, analytics",Seattle,Geekwire,Private,NaN
3,Addepar,20,3,1/18/2023,"fintech, data analytics","Mountain View, CA",Fortune,Private,NaN
4,Thirty Madison,24,3,5/13/2022,Health Care,New York,Axios,Private,"Per Axios, 22 of the employees laid off were f..."


# Preprocess

In [84]:
data.isnull().sum()

company                            0
total_layoffs                      0
impacted_workforce_percentage      0
reported_date                      0
industry                           0
headquarter_location               0
sources                            0
status                             0
additional_notes                 467
dtype: int64

ada sebanyak 467 data kosong di "additional_notes", drop aja ga penting juga wkwk

In [85]:
data.drop('additional_notes', inplace=True, axis=1)

In [86]:
data.head()

,company,total_layoffs,impacted_workforce_percentage,reported_date,industry,headquarter_location,sources,status
0,C2FO,20,2,12/9/2022,"Fintech, payments","Leawood, KS",Kansas City Business Journal,Private
1,VideoAmp,Unclear,2,9/19/2022,Advertising platforms,Los Angeles,Company memo,Private
2,Amperity,13,3,8/20/2022,"Saas, analytics",Seattle,Geekwire,Private
3,Addepar,20,3,1/18/2023,"fintech, data analytics","Mountain View, CA",Fortune,Private
4,Thirty Madison,24,3,5/13/2022,Health Care,New York,Axios,Private


In [87]:
data.columns

Index(['company', 'total_layoffs', 'impacted_workforce_percentage',
       'reported_date', 'industry', 'headquarter_location', 'sources',
       'status'],
      dtype='object')

In [88]:
dn = data['sources'].value_counts()
dn

TechCrunch          52
The Information     23
Business Insider    23
Calcalistech        20
LinkedIn            16
                    ..
CityWire             1
OneTrust             1
Mobi Health News     1
Homeward             1
Doma                 1
Name: sources, Length: 221, dtype: int64

based on sources, most of the data acquired from TechCrunch

In [89]:
industry_counts = data.groupby('industry').size().reset_index(name='count').sort_values(by='count', ascending=False)
industry_counts

,industry,count
91,Fintech,24
109,Health Care,17
166,PropTech,15
63,E-commerce,13
52,Cybersecurity,10
...,...,...
121,"IT, future of work",1
122,"IT, software",1
125,"Insurance, fintech",1
126,Insurtech,1


ternyata golongan paling banyak layoff ada di industri Fintech

berhubung namanya tidak seragam dan ada simbol, saatnya menyelaraskan nama tersebut

In [90]:
ind = []
for i in data['industry'].to_list():
    ind.append(i.split(' ')[-1])

ind = [x.lower() for x in ind]
ind = [x.replace("-", "") for x in ind]

data['industry'] = ind

In [91]:
data['industry'].head(10)

0         payments
1        platforms
2        analytics
3        analytics
4             care
5             tech
6    cybersecurity
7        ecommerce
8         wellness
9             tech
Name: industry, dtype: object

In [92]:
industry_counts = data.groupby('industry').size().reset_index(name='count').sort_values(by='count', ascending=False)
industry_counts

,industry,count
53,fintech,35
107,software,32
94,proptech,30
40,ecommerce,29
20,care,27
...,...,...
49,events,1
46,esignature,1
44,energy,1
42,editing,1


In [93]:
data['industry'].head(10).to_list()

['payments',
 'platforms',
 'analytics',
 'analytics',
 'care',
 'tech',
 'cybersecurity',
 'ecommerce',
 'wellness',
 'tech']

lakukan hal yang sama pada kolom HQ

In [94]:
data['headquarter_location'].head()

0          Leawood, KS
1          Los Angeles
2              Seattle
3    Mountain View, CA
4             New York
Name: headquarter_location, dtype: object

In [96]:
data['headquarter_location'] = data['headquarter_location'].str.lower().str.replace("-", '').str.replace(" ", '').str.split(',').str[-1]

jadi ini bisa juga digunakan dari pada sebelumnya yang harus ngasih array kosong dahulu.

In [97]:
data['headquarter_location'].head()

0            ks
1    losangeles
2       seattle
3            ca
4       newyork
Name: headquarter_location, dtype: object

semua data udah seragam sekarang

In [98]:
data.head(10)

,company,total_layoffs,impacted_workforce_percentage,reported_date,industry,headquarter_location,sources,status
0,C2FO,20,2,12/9/2022,payments,ks,Kansas City Business Journal,Private
1,VideoAmp,Unclear,2,9/19/2022,platforms,losangeles,Company memo,Private
2,Amperity,13,3,8/20/2022,analytics,seattle,Geekwire,Private
3,Addepar,20,3,1/18/2023,analytics,ca,Fortune,Private
4,Thirty Madison,24,3,5/13/2022,care,newyork,Axios,Private
5,NextRoll,Unclear,3,7/8/2022,tech,sanfrancisco,Business Insider,Private
6,Armis Security,25,4,12/8/2022,cybersecurity,sanfrancisco,Calcalistech,Private
7,Foxtrot,26,4,9/23/2022,ecommerce,chicago,Chicago Business,Private
8,Headspace Health,50,4,12/14/2022,wellness,losangeles,Bloomberg,Private
9,RealSelf,11,5,7/21/2022,tech,seattle,GeekWire,Private


# EDA